In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time
import pandas as pd

In [38]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)

# 메인 홈페이지로 이동
driver.get("https://www.koreabaseball.com/")

# 기록 페이지로 이동
css_selector = "#lnb > li:nth-child(3)"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

# 기록실 페이지로 이동
css_selector = "#lnb02"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

C:\Users\Luna\AppData\Local\Temp\ipykernel_16716\1968520446.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)


In [39]:
select = Select(driver.find_element(By.NAME, "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"))
select.select_by_value(value='OB')

time.sleep(3)

In [40]:
thead_xpath = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/table/thead/tr'
tbody_xapth = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/table/tbody'

table_head = driver.find_element(By.XPATH, thead_xpath).text.split(' ')
table_value = driver.find_element(By.XPATH, tbody_xapth).text.split('\n')

In [41]:
# 2페이지 이동
id_selector = 'cphContents_cphContents_cphContents_ucPager_btnNo2'
elem = driver.find_element(By.ID, id_selector)
elem.click()

time.sleep(3)

In [42]:
temp = driver.find_element(By.XPATH, tbody_xapth).text.split('\n')
table_value.extend(temp)

In [43]:
record_values = []

for item in table_value:
    record_values.append(item.split(' '))

doosan_dataframe = pd.DataFrame(columns=table_head)
for i in range(0, len(record_values)):
    doosan_dataframe.loc[i] = record_values[i]

doosan_dataframe.to_excel('test.xlsx', index=False)